In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [ ]:
matplotlib.rcParams['figure.figsize'] = [10, 5]

Here we will explain what we can do when we deal with categorical variables. That is variables that are not numeric.

## Labelizer

The easiest way to deal with them is to just transform them into numbers. Let's look at the following example.

__Example__ Let's assume that one of our column contains names of animals. There are three of them _dog_, _cat_ and _horse_. We can then assign them numbers: 0, 1 and 2. Hence we have transformed categorical variable into numeric one. Let's look how we can do this in `python`.

So let's put all out classes as a list.

In [ ]:
categories = ["dog", "cat", "horse"]

Now, our categories have a numeration. That is on order in the list. So we can easily create a dictionary that transform an animal name into a number:
* dog to 0
* cat to 1
* horse to 2.

In python it can be done as follows.

In [ ]:
col2num = {categories[i]: i for i in range(len(categories))}

Now let's see how we can transform a column of a `DataFrame` using this dictionary.

In [ ]:
animals_df = pd.DataFrame({"name": ["dog", "dog", "cat", "dog", "horse", "cat", "dog"]})
animals_df["name_label"] = animals_df["name"].apply(lambda name: col2num[name])
animals_df

The most important problem with this kind of encoding is that there is no natural order of those categories. There is reason why _cat_ should go between _dog_ and _horse_. This could cause a problem when we try to do linear regression. We will see it in the following example.

### Example

Let's train a model of prices of houses using `Neighborhood` variable. 

In [ ]:
houses = pd.read_csv("data/house-prices-advanced-regression-techniques/train.csv", index_col="Id")

target_col = "SalePrice"
categorical_column = "Neighborhood"
Xy = houses[[categorical_column, target_col]]
X = Xy.drop(target_col, axis=1)
y = Xy[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

In [ ]:
classes = X[categorical_column].unique()
col2num = {classes[i]: i for i in range(len(classes))}
X_train[categorical_column + "_label"] = X_train[categorical_column].apply(lambda cls: col2num[cls])
X_dev[categorical_column + "_label"] = X_dev[categorical_column].apply(lambda cls: col2num[cls])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression()
reg.fit(X_train[[categorical_column + "_label"]], y_train)
y_dev_hat = reg.predict(X_dev[[categorical_column + "_label"]])
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

So let's see why adding sort of random numbers to categories is not the best idea. We will plot the prediction and labels.

In [ ]:
# Ploting real data y (blue)
plt.scatter(X_dev[[categorical_column + "_label"]], y_dev, alpha=0.3, c="blue")
# versus predicted y_hat (red)


plt.plot(X_dev[[categorical_column + "_label"]], y_dev_hat, alpha=0.3, c="red")

plt.xlabel("Labelized category")
plt.ylabel("Sale Price")

We see that prediction is almost constant and we probably should reorder labels in a different ways. There two ways of solving it that we will discuss here.
1. We could labelized categories in a way that they do not have an order. We can do this using One-Hot-Encoding.
2. Labelized categories by numbers that makes sense.

## One-Hot-Encoding 

Let's comeback to animals example. The way to labelized categories in the way that they will not have any order is called One-Hot-Encoding. It is done by creating that many columns as categories we have and fill them with zeros, except the one that corresponds the category. So encode categories as follows:
* dog to `[1, 0, 1]`
* cat to `[0, 1, 0]`
* horse to `[0, 0, 1]`.

In [ ]:
def label_one_hot_encoding(series, dict_length=None):
    if dict_length is None:
        dict_length = len(series.unique())
    return np.eye(dict_length)[series]

print(animals_df)
print(label_one_hot_encoding(animals_df["name_label"], len(categories)))

Now, let's train it and see it it's get better.

In [ ]:
dict_length = len(X_train[categorical_column + "_label"].unique())

In [ ]:
reg.fit(label_one_hot_encoding(X_train[categorical_column + "_label"], dict_length), y_train)
y_dev_hat = reg.predict(label_one_hot_encoding(X_dev[categorical_column + "_label"], dict_length))
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

## Value encoding

Another way of encoding categories is to use average values of target variable. Actually in this situation we will get exactly the same predictions `y_dev_hat`. The most important think to remember is to use `X_train` and `y_train`. Let's see with the following code what it means.

In [ ]:
category_value_df = X_train[[categorical_column]]
category_value_df[categorical_column+"_value"] = y_train
category_value_df = category_value_df.groupby(categorical_column, as_index=False).mean()
category_value_df.head()

In [ ]:
y_dev_hat_value_encoded = \
    X_dev.merge(category_value_df, how="left", on=categorical_column)[[categorical_column+"_value"]]
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

## Putting all together

In [ ]:
columns = ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 
'TotalBsmtSF', "Neighborhood", "ExterQual"]

target_col = "SalePrice"
X = houses[columns]
y = houses[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

In [ ]:
class Labelizer:
    
    def __init__(self, categories):
        self.categories = categories
        self.cat2num = {}
        self.num2cat = {}
        
    def generate_dicts(self):
        self.cat2num = {self.categories[i]: i for i in range(len(self.categories))}
        self.num2cat = {i: self.categories[i] for i in range(len(self.categories))}
        
    def fit(self):
        self.generate_dicts()
        return self
        
    def transform(self, series):
        return series.apply(lambda cat: self.cat2num[cat])
    
labelizer_Neighborhood = Labelizer(X_train["Neighborhood"].unique()).fit()  
labelizer_ExterQual = Labelizer(X_train["ExterQual"].unique()).fit()

def labelize_X(X):
    labelized_X = X.drop(["Neighborhood", "ExterQual"], axis=1)
    labelized_X["Neighborhood_labelized"] = labelizer_Neighborhood.transform(X["Neighborhood"])
    labelized_X["ExterQual_labelized"] = labelizer_ExterQual.transform(X["ExterQual"])
    return labelized_X

X_train_labelized = labelize_X(X_train)
X_dev_labelized = labelize_X(X_dev)

reg.fit(X_train_labelized, y_train)
y_dev_hat = reg.predict(X_dev_labelized)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
A = np.array([[1, 2], [3, 4], [5, 6]])
B = np.array([[7], [8], [9]])
C = np.array([[11, 12], [13, 14], [16, 15]])
np.concatenate([A, B, C], axis=1)

In [ ]:
class OneHotEncoderWithLabelizer:
    
    def __init__(self, categories):
        self.categories = categories
        self.labelizer = Labelizer(categories)
            
    def fit(self):
        self.labelizer.fit()
        return self
        
    def transform(self, series):
        series_labelized = self.labelizer.transform(series)
        return np.eye(len(self.categories))[series_labelized]
    
one_hot_encoder_Neighborhood = OneHotEncoderWithLabelizer(X_train["Neighborhood"].unique()).fit()  
one_hot_encoder_ExterQual = OneHotEncoderWithLabelizer(X_train["ExterQual"].unique()).fit()

def one_hot_encode_X(X):
    X_other = X.drop(["Neighborhood", "ExterQual"], axis=1).as_matrix()
    neighborhood_encoded = one_hot_encoder_Neighborhood.transform(X["Neighborhood"])
    exterQual_encoded = one_hot_encoder_ExterQual.transform(X["ExterQual"])
    return np.concatenate([X_other, neighborhood_encoded, exterQual_encoded], axis=1)

X_train_encoded = one_hot_encode_X(X_train)
X_dev_encoded = one_hot_encode_X(X_dev)

reg.fit(X_train_encoded, y_train)
y_dev_hat = reg.predict(X_dev_encoded)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
import string

class ValueEncoder:
    
    def __init__(self, cat_colname=None, value_colname=None, agg_function='mean',):
        self.agg_function = agg_function
        self.cat_colname = cat_colname if cat_colname is not None else\
            ''.join(np.random.choice(list(string.ascii_lowercase + string.digits)) for _ in range(10))
        self.value_colname = value_colname if value_colname is not None else\
            self.cat_colname + "_" + self.agg_function
        self.cat_value_df = None
            
    def fit(self, cat_series, value_series):
        cat_value_df = pd.DataFrame({
            self.cat_colname: cat_series,
            self.value_colname: value_series
        })
        self.cat_value_df = cat_value_df\
            .groupby(self.cat_colname, as_index=False)\
            .agg(self.agg_function)
        return self
        
    def transform(self, cat_series):
        return pd.DataFrame({self.cat_colname:cat_series})\
            .merge(self.cat_value_df, on=self.cat_colname, how="left")[self.value_colname]

value_encoder_Neighborhood = ValueEncoder("Neighborhood")\
   .fit(X_train["Neighborhood"], y_train)  
value_encoder_ExterQual = ValueEncoder("ExterQual")\
   .fit(X_train["ExterQual"], y_train)

def value_encode_X(X):
    encoded_X = X.drop(["Neighborhood", "ExterQual"], axis=1).reset_index(drop=True)
    encoded_X["Neighborhood_encoded"] = value_encoder_Neighborhood.transform(X["Neighborhood"])
    encoded_X["ExterQual_encoded"] = value_encoder_ExterQual.transform(X["ExterQual"])
    return encoded_X

X_train_encoded = value_encode_X(X_train)
X_dev_encoded = value_encode_X(X_dev)
reg.fit(X_train_encoded, y_train)
y_dev_hat = reg.predict(X_dev_encoded)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

In [ ]:
colname = "LotShape"
classes = X[colname].unique()
col2num = {classes[i]: i for i in range(len(classes))}
X_train[colname + "_int"] = X_train[colname].apply(lambda cls: col2num[cls])
X_dev[colname + "_int"] = X_dev[colname].apply(lambda cls: col2num[cls])

In [ ]:
def label_one_hot_encoding(series, dict_length=None):
    if dict_length is None:
        dict_length = len(series.unique())
    return np.eye(dict_length)[series]
    
label_one_hot_encoding(animals_df["name_label"], len(categories))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
def test_column(colname):
    classes = X[colname].unique()
    col2num = {classes[i]: i for i in range(len(classes))}
    X_train[colname + "_int"] = X_train[colname].apply(lambda cls: col2num[cls])
    X_dev[colname + "_int"] = X_dev[colname].apply(lambda cls: col2num[cls])
    reg = LinearRegression()
    reg.fit(X_train[[colname + "_int"]], y_train)
    y_dev_hat = reg.predict(X_dev[[colname + "_int"]])
    print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

for colname in categorical_columns:
    print("----------------")
    print(colname)
    test_column(colname)


In [ ]:
def test_column(colname):
    classes = X[colname].unique()
    col2num = {classes[i]: i for i in range(len(classes))}
    X_train[colname + "_int"] = X_train[colname].apply(lambda cls: col2num[cls])
    X_dev[colname + "_int"] = X_dev[colname].apply(lambda cls: col2num[cls])
    reg = LinearRegression()
    reg.fit(label_one_hot_encoding( X_train[colname + "_int"], dict_length=len(classes)), y_train)
    y_dev_hat = reg.predict(label_one_hot_encoding(X_dev[colname + "_int"], dict_length=len(classes)))
    print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

for colname in categorical_columns:
    print("----------------")
    print(colname)
    test_column(colname)

In [ ]:
matplotlib.rcParams['figure.figsize'] = [20, 10]

In [ ]:
data = pd.read_csv("data/titanic/train.csv", index_col="PassengerId")

In [ ]:
data.head()

In [ ]:
X = data.drop("Survived", axis=1)
y = data["Survived"]

In [ ]:
X.isna().sum()

In [ ]:
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

## Survived based on sex

In [ ]:
sex_df = X_train[["Sex", "Name"]].copy()
sex_df["Survived"] = y_train

sex_df.reset_index().groupby(["Sex", "Survived"]).count()

In [ ]:
sexes = ["male", "female"]
sex2num = {sexes[i]: i for i in range(len(sexes))}

sex_encoded = X_dev["Sex"].apply(lambda sex: sex2num.get(sex))

In [ ]:
print("Accuracy :", accuracy_score(y_dev, y_dev_hat))
print("f1-score: ", f1_score(y_dev, sex_encoded))

## Passenger class

In [ ]:
X_train.groupby("Pclass")["Name"].count()

In [ ]:
def sex_pclass_transform(X):
    new_X = X[['Pclass']].copy()
    new_X = X[['SibSp']].copy()
    #new_X["sex"] = X["Sex"].apply(lambda sex: sex2num.get(sex))
    return new_X

from sklearn.tree import DecisionTreeClassifier, export_graphviz

clf = DecisionTreeClassifier()
clf.fit(sex_pclass_transform(X_train), y_train)
y_dev_hat = clf.predict(sex_pclass_transform(X_dev))
print("Accuracy :", accuracy_score(y_dev, y_dev_hat))
print("f1-score: ", f1_score(y_dev, sex_encoded))

In [ ]:
import graphviz 

dot_data = export_graphviz(
    clf, out_file=None, 
    feature_names=sex_pclass_transform(X_train).columns.values,  
    class_names=["Not survived", "survived"],  
    filled=True, rounded=True,  
    special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(sex_pclass_transform(X_train), y_train)
y_dev_hat = clf.predict(sex_pclass_transform(X_dev))
print("Accuracy :", accuracy_score(y_dev, y_dev_hat))
print("f1-score: ", f1_score(y_dev, sex_encoded))

In [ ]:
X_train["SibSp"].unique()

In [ ]:
one_hot_targets = np.eye(nb_classes)[targets]

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)